In [1]:
import re
from itertools import chain
from tqdm import tqdm
import numpy as np
import collections

DICT = []

STATE_WORD = 0#статус строки (слово)
STATE_TYPE = 1
STATE_MEANING_START = 2

class DictProcessor():
    def __init__(self): #конструктор
        self.word = ""
        self.meaning = ""
        
        self.state = STATE_WORD
        
        self.dict = []

    def update(self, line):#определяем тип строки
        if line == "":
            self.go(STATE_WORD)
            return    
        elif re.match(r"^\s*\d+\.(.*)", line):
            m = re.match(r"^\s*\d+\.(.*)", line)
            line = m.group(1)
            self.go(STATE_TYPE)
        elif re.match(r"^\s*\d+\)", line):   
            m = re.match(r"^\s*\d+\)(.*)", line)
            line = m.group(1)
            self.go(STATE_MEANING_START)
        
        self._process(line)
        
    def _process(self, line):#сканируем строчку
        if self.state == STATE_WORD:
            self.word = line
            self.go(STATE_TYPE)
        elif self.state == STATE_TYPE:
            pass
            self.go(STATE_MEANING_START)
        elif self.state == STATE_MEANING_START:
            self.meaning += " " + line.strip()
    
    def _publish(self):#записываем в память
        self.meaning = self.meaning.strip()
        if self.word != "" and self.meaning != "" and len(self.word) >= 3:
            if "см. " in self.meaning or ":" in self.meaning or "*" in self.meaning: 
                pass
            else:
                self.dict.append((self.word, self.meaning))
            
            #print("!!!| [%s] - [%s]" % (self.word, self.meaning))        
        self.meaning = ""                        
    
    def go(self, new_state):#если меняем статус с: то публикуем
        if self.state == STATE_MEANING_START:
            self._publish()
        
        self.state = new_state   
        
    def post_process(self):#в помойку многозначные слова
        counts = collections.Counter((w_m[0] for w_m in self.dict))
        
        self.dict = list(filter(lambda w_m: counts[w_m[0]] <= 1, self.dict))        

processor = DictProcessor()        
        
with open("efremova.txt") as fin:#читаем по строчно файл
    row = -1
    
    state = STATE_WORD
    
    word = ""
    for line in fin:
        row += 1
        
        if row < 3:
            continue
            
        if row > 100 and False:
            break
        
        line = line.strip()                
        
        #print("%d: %s" % (processor.state, line))
        
        processor.update(line)                        
        
processor.post_process()        

In [2]:
print(len(processor.dict))

49674


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer#текст в фичи

In [4]:
NGRAMS = (2,4)
words_tr = TfidfVectorizer(analyzer='char', ngram_range=NGRAMS, use_idf=False)#настраиваем анализатор
f_words = words_tr.fit_transform((w_m[0] for w_m in processor.dict))#матрица слова/нграммам состоит из чисел кол-во нграмм в слове/кол-во слов с этой нграммой

In [5]:
import pymorphy2 #нормализатор
morph = pymorphy2.MorphAnalyzer()
cache = {}

def preprocess(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0].normal_form
        
    return cache[word]    

def tokenizer(line):
    res = []
    for w in re.findall("\w+", line):
        res.append(preprocess(w))
        
    return res    

meaning_tr = TfidfVectorizer(tokenizer=tokenizer)
f_meaning = meaning_tr.fit_transform((w_m[1] for w_m in processor.dict))#аналогично определения / слова

In [6]:
for i, (k, v) in enumerate(cache.items()):
    print("%s-%s" % (k, v))
    if i > 20:
        break

старинная-старинный
мелкая-мелкий
серебряная-серебряный
монета-монета
имевшая-иметь
хождение-хождение
на-на
кавказе-кавказ
а-а
также-также
аббатисса-аббатисса
настоятельница-настоятельница
католического-католический
женского-женский
монастыря-монастырь
слово-слово
образованное-образовать
из-из
первых-один
букв-буква
единиц-единица
в-в


In [7]:
for i, w in enumerate(meaning_tr.get_feature_names()):
    if i > 100:
        print(w)
    if i > 300:
        break

206
21
212
22
23
24
2400
244
25
2500
27
28
2а1
2а2
3
30
300
3000
32
335
34
35
36
366
4
40
400
4047
44
45
450
46
5
50
500
58
6
60
6000
7
70
79
8
80
81
83
86
9
90
91
b
c
i
ii
iii
iv
ix
j
p
pluralia
s
tantum
v
vi
vii
viii
x
xi
xii
xiii
xiv
xix
xv
xvi
xvii
xviii
xx
а
абак
аббатисса
аббревиатура
абвер
абзац
абиссаль
абиссинец
абиссиния
аболиционизм
абонемент
абориген
абразив
абразивный
абрикос
абрикосовый
абрис
абсолют
абсолютизм
абсолютно
абсолютный
абсорбция
абстрактный
абстракционизм
абстракция
абсурд
абсцесс
авангардизм
авангардистский
аванс
авансом
авантюра
авантюрист
авар
авария
август
авенариус
авиабензин
авиабомба
авиадвигатель
авиадесант
авиазавод
авиакомпания
авиалиния
авиамодель
авиамодельный
авиамотор
авианосец
авиапассажир
авиапочта
авиаприбор
авиапромышленность
авиаразведка
авиасвязь
авиасклад
авиастроение
авиатор
авиатранспорт
авиатрасса
авиаучилище
авиационный
авиация
авиачасть
авитаминоз
аврал
австерия
австралийский
австралия
австриец
австрия
австро
авто
автобиография
автоб

In [8]:
print(f_meaning.shape)#слова - фичи (кол-во)

(49674, 33414)


In [9]:
print(f_meaning[2, :])

  (0, 26635)	0.427375860879
  (0, 16203)	0.137861447057
  (0, 8395)	0.17046590785
  (0, 16552)	0.185800592885
  (0, 2038)	0.146346995168
  (0, 6655)	0.106179766237
  (0, 2178)	0.0503729459541
  (0, 26646)	0.377326028937
  (0, 28460)	0.16819436264
  (0, 3688)	0.188663014468
  (0, 8321)	0.232342387504
  (0, 29072)	0.222256468954
  (0, 19291)	0.222315551105
  (0, 14472)	0.172945381572
  (0, 31879)	0.0951766995878
  (0, 5397)	0.116218656405
  (0, 1739)	0.124872299645
  (0, 22491)	0.20767366893
  (0, 10210)	0.188663014468
  (0, 8685)	0.068576941428
  (0, 21516)	0.134181367753
  (0, 25806)	0.0992357073551
  (0, 26651)	0.186380117027
  (0, 14469)	0.138948585788
  (0, 25241)	0.0662660813351
  (0, 6483)	0.0989023438953
  (0, 24937)	0.215532485378
  (0, 6414)	0.215532485378


In [10]:
#load tf info
from collections import defaultdict#словарик со знач по умолчанию

words_tf = defaultdict(lambda: 0)
with open("tf.txt") as fin:#частотный словарик
    for line in fin:
        tf, word = line.strip().split()
        tf = int(tf)
        
        words_tf[preprocess(word)] += tf
        
f_words_tf = np.fromiter((words_tf[w_m[0]] for w_m in processor.dict), dtype=int)
print(f_words_tf.shape)      

(49674,)


In [11]:
"""
words_tf = None
cache = None

import gc
gc.collect()
"""

'\nwords_tf = None\ncache = None\n\nimport gc\ngc.collect()\n'

In [12]:
BAD_W = 10

from scipy.sparse import vstack, hstack#нужно для склейки матриц
from random import randint

X = []#строчки с парами
Y = []#правильность пары
W = []#веса

for i in tqdm(range(f_words.shape[0] // 1)):
    f_w = f_words[i:i+1, :]#матрица равная i- ой строке (строка - слово)
    f_m = f_meaning[i:i+1, :]
        
    Y.append(1)
    W.append(BAD_W)
    X.append(hstack([f_w, f_m]))
    for j in range(BAD_W):#накидываем неправильные пары
        while True:
           rand_i = randint(0, f_words.shape[0] - 1)
           if rand_i != i:
                break
        Y.append(0)  
        W.append(1)
        X.append(hstack([f_w, f_meaning[rand_i:rand_i+1, :]]))
        
    #break    
X = vstack(X)# преобразовываем тип данных    
Y = np.array(Y)
W = np.array(W)

100%|██████████| 49674/49674 [03:34<00:00, 231.04it/s]


In [13]:
print(X.shape)
print(Y.shape)
print(f_words.shape)
print(f_meaning.shape)

(546414, 88063)
(546414,)
(49674, 54649)
(49674, 33414)


In [17]:
from sklearn.model_selection import train_test_split#"рандомно" делит Х на две части
X_train, X_test, y_train, y_test, W_train, W_test = train_test_split(X, Y, W, test_size=0.1, random_state=42)

In [18]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [19]:
t = "svm"

if t == "svm":
    svm = LinearSVC()
    clf = CalibratedClassifierCV(svm) #призываем свм с вероятностью
    
clf.fit(X_train, y_train, sample_weight=W_train)#тренеровка

y_proba = clf.predict_proba(X_test)#предсказание

In [21]:
from sklearn.metrics import roc_auc_score
print(y_proba)
print(y_test)
print(roc_auc_score(y_test, y_proba[:, 1], sample_weight=W_test))#~количество угадываний / попыток (какие мы крутые)#насколько

[[ 0.37569052  0.62430948]
 [ 0.65043025  0.34956975]
 [ 0.3206458   0.6793542 ]
 ..., 
 [ 0.62380648  0.37619352]
 [ 0.65181981  0.34818019]
 [ 0.68842463  0.31157537]]
[1 0 1 ..., 0 0 0]
0.939735340869


In [22]:
y_res = clf.predict_proba(X)[:, 1]#[1::BAD_W + 1]#применим ко все парам 
y_test = Y#[1::BAD_W + 1]

In [29]:
#0.9 0.2 0.4 0.5
#1 1 0 0

freq = np.repeat(f_words_tf.astype(float) / f_words_tf.max(), BAD_W + 1)#нормализуем частоты
freq[np.repeat(f_words_tf, BAD_W + 1) == 0] = 1.0#слова у которых нет частоты

#freq = 0.0

print(freq)

diff = np.abs(y_res - y_test) * (1.0 - freq)#тест это 0 или 1 рез вероятность
diff_order = np.argsort(diff)[::-1] #diff[diff_order] - decreasing#сортируем массив

N = 100

#print(diff[diff_order])
print(diff[diff_order[:N]])
print(y_res[diff_order[:N]])
print(y_test[diff_order[:N]])

docs_nums = diff_order[:N] // (1 + BAD_W)#переходим к индексам с правильными определениями
print(docs_nums)

for i in docs_nums:
    print("%s - %s" % processor.dict[i])

[ 1.  1.  1. ...,  1.  1.  1.]
[ 0.95780011  0.95058981  0.94543882  0.94312293  0.93644607  0.93130718
  0.93001454  0.92514843  0.92153525  0.91968165  0.91826692  0.91818729
  0.91666048  0.91564434  0.91293816  0.91186926  0.91092197  0.91065428
  0.91038235  0.91033974  0.90998015  0.9001769   0.89839505  0.89775588
  0.89705512  0.89600671  0.89325857  0.89071144  0.88995622  0.88922751
  0.88814182  0.88773173  0.8874029   0.88714457  0.8859009   0.88329262
  0.88254276  0.88184903  0.88088515  0.87999342  0.87922292  0.87908452
  0.87894399  0.87879032  0.87751796  0.87599352  0.87582071  0.87486606
  0.87480162  0.87452146  0.87422955  0.87409146  0.8736729   0.87347141
  0.8729408   0.87277292  0.8726089   0.87224981  0.87220492  0.87098195
  0.8704495   0.86976563  0.86954575  0.8688532   0.86867935  0.86860192
  0.86828811  0.86721071  0.86720575  0.86718877  0.86704453  0.86576962
  0.86396149  0.86370426  0.86320697  0.86267003  0.86262367  0.86180735
  0.86171554  0.8616

In [ ]:
мигач - Тот, кто заигрывает с девушками, игриво подмигивает им.